# Webscraping The Texas Department of Criminal Justice's website

I stumbled upon this website and was interested to see a full list of last statements made by criminals who faced capital punishment, however, there were 569 in total and I definitely didn't want to sort through that. Thus, this project came about and the final goal was to have a csv file with all of the information and full last statements from all of Texas' executed offenders.
From there, the data can be used in anyway to analyze the profiles of the executed offenders. Unfortunately the file does not contain the type of crime commited for all of the offenders as more than two-thirds of the information was uploaded as a .jpg (which should be my next project). 

In [59]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# important libraries for formatting and creating a delay.
import unicodedata
import time

# To display non-truncated dataframe information
pd.set_option('display.max_colwidth', -1)

#### Start by cooking the soup:

In [94]:
source = requests.get('https://www.tdcj.texas.gov/death_row/dr_executed_offenders.html').text
soup = BeautifulSoup(source, 'lxml')

#### Isolating the HTML to locate the class 'overflow' which contains all of the information we need. 
I've included a snippet of the output since the actual one was too long.

In [95]:
match = soup.find('div', class_='overflow')

#### Pulling out all of the headers of the table.

In [61]:
for headers in match.find_all('th'):
    print(headers.text)

Execution
Link
Link
Last Name
First Name
TDCJNumber
Age
Date
Race
County


#### First attempt on pulling out multiple 'td' tags

In [6]:
# clearly this doesn't work

for prisoner_info in match.tr.th.text:
    print(prisoner_info)

E
x
e
c
u
t
i
o
n


#### Getting the prisoner info link and last statement link for each executed prisoner and putting them into a useable array.

In [96]:
links_array = []

for link in match.find_all('a'):
    testing1 = link.get('href')
    link = f'https://www.tdcj.texas.gov/death_row/{testing1}'
    links_array.append(link)
print(links_array[0:10])

['https://www.tdcj.texas.gov/death_row/dr_info/ochoaabel.html', 'https://www.tdcj.texas.gov/death_row/dr_info/ochoaabellast.html', 'https://www.tdcj.texas.gov/death_row/dr_info/gardnerjohn.html', 'https://www.tdcj.texas.gov/death_row/dr_info/gardnerjohnlast.html', 'https://www.tdcj.texas.gov/death_row/dr_info/runnelstravis.html', 'https://www.tdcj.texas.gov/death_row/dr_info/runnelstravislast.html', 'https://www.tdcj.texas.gov/death_row/dr_info/halljusten.html', 'https://www.tdcj.texas.gov/death_row/dr_info/halljustenlast.html', 'https://www.tdcj.texas.gov/death_row/dr_info/sparksrobert.html', 'https://www.tdcj.texas.gov/death_row/dr_info/sparksrobertlast.html']


#### You will see that I like to do a lot of checks, one of them is below:
The array should be of length 1,138 as there are 569 executed offenders and each link alternates between a link to the offender's information and the offender's last statement

In [9]:
# checking (569 x 2 = 1,138)
len(links_array)

1138

#### Separating the array into two separate prisoner info and last statements lists:
An even link index # corresponds to prisoner info, an odd link index # corresponds to a last statement.

In [120]:
#Even
prisoner_info = links_array[::2]
prisoner_info[0:10]

['https://www.tdcj.texas.gov/death_row/dr_info/ochoaabel.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/gardnerjohn.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/runnelstravis.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/halljusten.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/sparksrobert.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/solizmarkanthony.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/crutsingerbilly.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/swearingenlarry.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/kingjohn.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/_coble.jpg']

In [121]:
# Odd
last_statement_links = links_array[1::2]
last_statement_links[0:10]

['https://www.tdcj.texas.gov/death_row/dr_info/ochoaabellast.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/gardnerjohnlast.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/runnelstravislast.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/halljustenlast.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/sparksrobertlast.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/solizmarkanthonylast.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/crutsingerbillylast.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/swearingenlarrylast.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/kingjohnlast.html',
 'https://www.tdcj.texas.gov/death_row/dr_info/coblebillielast.html']

In [65]:
len(prisoner_info)

569

In [66]:
len(last_statement_links)

569

#### Creating a dataframe with the information on the website and saving it into a csv file:

Fortunately pandas makes it easy to parse information from a website.

In [103]:
url = 'https://www.tdcj.texas.gov/death_row/dr_executed_offenders.html'
html = requests.get(url).content
df = pd.read_html(html)
print(df)
df[0].to_csv(r'/Users/stephanie/Desktop/prison-env/xecuted_offenders.csv', index = False, header=True)
#kept recieving a 'list' object has no attribute 'to_csv' error message so had to specify df[0] even though there is only one df on the page

[     Execution                  Link          Link.1       Last Name  \
0    569        Offender Information  Last Statement  Ochoa            
1    568        Offender Information  Last Statement  Gardner          
2    567        Offender Information  Last Statement  Runnels          
3    566        Offender Information  Last Statement  Hall             
4    565        Offender Information  Last Statement  Sparks           
5    564        Offender Information  Last Statement  Soliz            
6    563        Offender Information  Last Statement  Crutsinger       
7    562        Offender Information  Last Statement  Swearingen       
8    561        Offender Information  Last Statement  King             
9    560        Offender Information  Last Statement  Coble            
10   559        Offender Information  Last Statement  Jennings         
11   558        Offender Information  Last Statement  Braziel, Jr.     
12   557        Offender Information  Last Statement  Garcia   

In [104]:
df2 = pd.read_csv('xecuted_offenders.csv')
df2.head(10)

,Execution,Link,Link.1,Last Name,First Name,TDCJNumber,Age,Date,Race,County
0,569,Offender Information,Last Statement,Ochoa,Abel,999450,47,2/6/2020,Hispanic,Dallas
1,568,Offender Information,Last Statement,Gardner,John,999516,64,1/15/2020,White,Collin
2,567,Offender Information,Last Statement,Runnels,Travis,999505,46,12/11/2019,Black,Potter
3,566,Offender Information,Last Statement,Hall,Justen,999497,38,11/6/2019,White,El Paso
4,565,Offender Information,Last Statement,Sparks,Robert,999542,45,9/25/2019,Black,Dallas
5,564,Offender Information,Last Statement,Soliz,Mark,999571,37,9/10/2019,Hispanic,Johnson
6,563,Offender Information,Last Statement,Crutsinger,Billy,999459,64,9/4/2019,White,Tarrant
7,562,Offender Information,Last Statement,Swearingen,Larry,999361,48,8/21/2019,White,Montgomery
8,561,Offender Information,Last Statement,King,John,999295,44,4/24/2019,White,Jasper
9,560,Offender Information,Last Statement,Coble,Billie,976,70,2/28/2019,White,McLennan


In [105]:
# checking the shape of the dataframe to make sure it's right.
df2.shape

(569, 10)

In [106]:
# Renaming some columns to get rid of any white spaces.

df2.rename(columns={'Last Name' : 'Last_Name','First Name' : 'First_Name'}, inplace = True)
df2.head(10)

,Execution,Link,Link.1,Last_Name,First_Name,TDCJNumber,Age,Date,Race,County
0,569,Offender Information,Last Statement,Ochoa,Abel,999450,47,2/6/2020,Hispanic,Dallas
1,568,Offender Information,Last Statement,Gardner,John,999516,64,1/15/2020,White,Collin
2,567,Offender Information,Last Statement,Runnels,Travis,999505,46,12/11/2019,Black,Potter
3,566,Offender Information,Last Statement,Hall,Justen,999497,38,11/6/2019,White,El Paso
4,565,Offender Information,Last Statement,Sparks,Robert,999542,45,9/25/2019,Black,Dallas
5,564,Offender Information,Last Statement,Soliz,Mark,999571,37,9/10/2019,Hispanic,Johnson
6,563,Offender Information,Last Statement,Crutsinger,Billy,999459,64,9/4/2019,White,Tarrant
7,562,Offender Information,Last Statement,Swearingen,Larry,999361,48,8/21/2019,White,Montgomery
8,561,Offender Information,Last Statement,King,John,999295,44,4/24/2019,White,Jasper
9,560,Offender Information,Last Statement,Coble,Billie,976,70,2/28/2019,White,McLennan


#### Converting lists to n x 1 dataframe:
To make it easier to concatenate these links to the original dataframe later.

In [107]:
info_df = pd.DataFrame(prisoner_info)
info_df.columns = ['Offender_Information_Link']
info_df.head(10)

,Offender_Information_Link
0,https://www.tdcj.texas.gov/death_row/dr_info/ochoaabel.html
1,https://www.tdcj.texas.gov/death_row/dr_info/gardnerjohn.html
2,https://www.tdcj.texas.gov/death_row/dr_info/runnelstravis.html
3,https://www.tdcj.texas.gov/death_row/dr_info/halljusten.html
4,https://www.tdcj.texas.gov/death_row/dr_info/sparksrobert.html
5,https://www.tdcj.texas.gov/death_row/dr_info/solizmarkanthony.html
6,https://www.tdcj.texas.gov/death_row/dr_info/crutsingerbilly.html
7,https://www.tdcj.texas.gov/death_row/dr_info/swearingenlarry.html
8,https://www.tdcj.texas.gov/death_row/dr_info/kingjohn.html
9,https://www.tdcj.texas.gov/death_row/dr_info/_coble.jpg


In [108]:
words_df = pd.DataFrame(last_statements)
words_df.columns = ['Last_Statement_Link']
words_df.head(10)

,Last_Statement_Link
0,https://www.tdcj.texas.gov/death_row/dr_info/ochoaabellast.html
1,https://www.tdcj.texas.gov/death_row/dr_info/gardnerjohnlast.html
2,https://www.tdcj.texas.gov/death_row/dr_info/runnelstravislast.html
3,https://www.tdcj.texas.gov/death_row/dr_info/halljustenlast.html
4,https://www.tdcj.texas.gov/death_row/dr_info/sparksrobertlast.html
5,https://www.tdcj.texas.gov/death_row/dr_info/solizmarkanthonylast.html
6,https://www.tdcj.texas.gov/death_row/dr_info/crutsingerbillylast.html
7,https://www.tdcj.texas.gov/death_row/dr_info/swearingenlarrylast.html
8,https://www.tdcj.texas.gov/death_row/dr_info/kingjohnlast.html
9,https://www.tdcj.texas.gov/death_row/dr_info/coblebillielast.html


In [109]:
# Removing unncessary columns
df2.drop(columns=['Link','Link.1'], inplace = True)
df2.head(10)

,Execution,Last_Name,First_Name,TDCJNumber,Age,Date,Race,County
0,569,Ochoa,Abel,999450,47,2/6/2020,Hispanic,Dallas
1,568,Gardner,John,999516,64,1/15/2020,White,Collin
2,567,Runnels,Travis,999505,46,12/11/2019,Black,Potter
3,566,Hall,Justen,999497,38,11/6/2019,White,El Paso
4,565,Sparks,Robert,999542,45,9/25/2019,Black,Dallas
5,564,Soliz,Mark,999571,37,9/10/2019,Hispanic,Johnson
6,563,Crutsinger,Billy,999459,64,9/4/2019,White,Tarrant
7,562,Swearingen,Larry,999361,48,8/21/2019,White,Montgomery
8,561,King,John,999295,44,4/24/2019,White,Jasper
9,560,Coble,Billie,976,70,2/28/2019,White,McLennan


In [110]:
# Saving the new dataframe.
df2.to_csv(r'/Users/stephanie/Desktop/prison-env/xecuted_offenders.csv', index = False, header=True)

In [22]:
# initializing the list
last_statement_list = []

#### The longest part to execute:
In this part of the code, I iterate through the list of links pulled from the website and grab the HTML from each link. After further inspection of the HTML, the paragraph 'p' tags were the ones that contained all of the pertinent information (the last statements of the offenders). I grabbed the text from those which were located at index[5]. Some of the last statements were empty or not encased a in 'p' tag at all so I had to use a try block to handle an IndexError that I got from some of the offenders.
I also set a delay of 2 seconds before another iteration occurs to prevent too many requests from being sent at once.

In [27]:
for link in last_statement_links:
    source1 = requests.get(link).text
    soup1 = BeautifulSoup(source1, 'lxml')
    try:
        looking = soup1.find_all('p')[5].text
        last_statement_list.append(looking)
        time.sleep(2)
        
    except IndexError:
        last_statement_list.append('No last statement.')
        time.sleep(2)

#### A satisfying list to look at.

In [112]:
last_statement_list[0:10]

['Yes sir. I  would like to thank God, my dad, my Lord Jesus savior for saving me and changing  my life. I want to apologize to my in-laws for causing all this emotional pain.  I love y’all and consider y’all my sisters I never had. I want to thank you for  forgiving me. Thank you warden. ',
 'Thank you thank you  where’s the family, ok I would like to say sorry for the grief I have caused. I  hope you find peace, joy and closure. Whatever it takes to forgive me. I am  sorry. I know you cannot forgive me but I hope one day you will. To my friends,  I love ya’ll. You have been good and such good influence people. I want to see  the Lord Jesus so bad. I hope you all understand. I am going to ask the Warden  to start. And ask Jesus to help them to forgive me one day please. I am very  sorry. For you guys I love you. Thank you Warden.',
 'None',
 'Yeah, I want to  address the Roundtree family and apologize for the pain and suffering I caused.  And to the Diaz’s family that I had to put you

#### Saving the df to a last_statements.csv file with a header "Last_Statement"

In [113]:
last_statement_df = pd.DataFrame(last_statement_list)
last_statement_df.columns = ['Last_Statement']
last_statement_df.to_csv(r'/Users/stephanie/Desktop/prison-env/last_statements.csv', index = False, header=True)

#### Being paranoid, I check again.

In [114]:
last_statement_df.shape

(569, 1)

####  Reading a random last statement:
Since the indeces are flipped, if you want to find the location of a specific person you will have to use the formula:
569 - (Offender Execution #) = iloc[x]

In [115]:
last_statement_df.iloc[362]

Last_Statement     I'd like to apologize to the victim's family. Ah, no ah, I really can't say, I don’t think I can say anything that will help, but I hope through your God, you can forgive me. I'm definitely not the person now that I was then. I was sick, afraid, and looking for love in all the wrong ways. I've caused you pain and grief beyond ever dreaming to cause someone of. I hope you will be able to forgive me. To my mother, I love you very much. Thanks, Jones.  
Name: 362, dtype: object

In [116]:
# Renaming a column.
df2.rename(columns={'Execution' : 'Execution_Number'}, inplace = True)
df2.head(10)

,Execution_Number,Last_Name,First_Name,TDCJNumber,Age,Date,Race,County
0,569,Ochoa,Abel,999450,47,2/6/2020,Hispanic,Dallas
1,568,Gardner,John,999516,64,1/15/2020,White,Collin
2,567,Runnels,Travis,999505,46,12/11/2019,Black,Potter
3,566,Hall,Justen,999497,38,11/6/2019,White,El Paso
4,565,Sparks,Robert,999542,45,9/25/2019,Black,Dallas
5,564,Soliz,Mark,999571,37,9/10/2019,Hispanic,Johnson
6,563,Crutsinger,Billy,999459,64,9/4/2019,White,Tarrant
7,562,Swearingen,Larry,999361,48,8/21/2019,White,Montgomery
8,561,King,John,999295,44,4/24/2019,White,Jasper
9,560,Coble,Billie,976,70,2/28/2019,White,McLennan


#### Appending Last_Statement and Offender_Information and full text of the last_statement to our main df:

In [117]:
df2 = pd.concat([df2, info_df, words_df, last_statement_df], axis=1)
df2.head(10)

,Execution_Number,Last_Name,First_Name,TDCJNumber,Age,Date,Race,County,Offender_Information_Link,Last_Statement_Link,Last_Statement
0,569,Ochoa,Abel,999450,47,2/6/2020,Hispanic,Dallas,https://www.tdcj.texas.gov/death_row/dr_info/ochoaabel.html,https://www.tdcj.texas.gov/death_row/dr_info/ochoaabellast.html,"Yes sir. I would like to thank God, my dad, my Lord Jesus savior for saving me and changing my life. I want to apologize to my in-laws for causing all this emotional pain. I love y’all and consider y’all my sisters I never had. I want to thank you for forgiving me. Thank you warden."
1,568,Gardner,John,999516,64,1/15/2020,White,Collin,https://www.tdcj.texas.gov/death_row/dr_info/gardnerjohn.html,https://www.tdcj.texas.gov/death_row/dr_info/gardnerjohnlast.html,"Thank you thank you where’s the family, ok I would like to say sorry for the grief I have caused. I hope you find peace, joy and closure. Whatever it takes to forgive me. I am sorry. I know you cannot forgive me but I hope one day you will. To my friends, I love ya’ll. You have been good and such good influence people. I want to see the Lord Jesus so bad. I hope you all understand. I am going to ask the Warden to start. And ask Jesus to help them to forgive me one day please. I am very sorry. For you guys I love you. Thank you Warden."
2,567,Runnels,Travis,999505,46,12/11/2019,Black,Potter,https://www.tdcj.texas.gov/death_row/dr_info/runnelstravis.html,https://www.tdcj.texas.gov/death_row/dr_info/runnelstravislast.html,None
3,566,Hall,Justen,999497,38,11/6/2019,White,El Paso,https://www.tdcj.texas.gov/death_row/dr_info/halljusten.html,https://www.tdcj.texas.gov/death_row/dr_info/halljustenlast.html,"Yeah, I want to address the Roundtree family and apologize for the pain and suffering I caused. And to the Diaz’s family that I had to put you through this, it should have never happened. And to my mom and Morelia I love you and I’m going to miss you all. I’m ready"
4,565,Sparks,Robert,999542,45,9/25/2019,Black,Dallas,https://www.tdcj.texas.gov/death_row/dr_info/sparksrobert.html,https://www.tdcj.texas.gov/death_row/dr_info/sparksrobertlast.html,"Umm, Pamela can you hear me Stephanie, Hardy, Marcus tell all the family I love them. I am sorry for the hard times and what hurts me is that I hurt y’all and um even for y’all too and Patricia she wrote me tell Patricia I wrote her back and to tell y’all what I said. I love y’all. I am ready"
5,564,Soliz,Mark,999571,37,9/10/2019,Hispanic,Johnson,https://www.tdcj.texas.gov/death_row/dr_info/solizmarkanthony.html,https://www.tdcj.texas.gov/death_row/dr_info/solizmarkanthonylast.html,"It’s 6:09 on September 10th, Kayla and David, I wanted to apologize for the grief and the pain that I caused y’all .I’ve been considering changing my life. It took me 27 years to do so. Man, I want to apologize, I don’t know if me passing will bring y’all comfort for the pain and suffering I caused y’all. I am at peace. I understand now the pain that I caused y’all man, I don’t know what else to say. It took a while to drag these years out. I am going with a humble heart. I made wrong decisions but, I forgave myself. I made a discussion not because of myself but because of everyone else. I forgave myself not for y’all but the pain I caused to my family."
6,563,Crutsinger,Billy,999459,64,9/4/2019,White,Tarrant,https://www.tdcj.texas.gov/death_row/dr_info/crutsingerbilly.html,https://www.tdcj.texas.gov/death_row/dr_info/crutsingerbillylast.html,"Hi ladies I wanted to tell ya’ll how much I love you. Thank you for being here for me. You have brought pleasure into my life in the short time I lived and known ya’ll. Ya’ll are very special not just to me but to the unit. There are so many lives that yall have touched over there that yall don’t even know about that guys talk about in the back .I am at peace now with and going to be with Jesus and my family. I am going to miss those pancakes and those old time black and white shows. Where I am going everything 

#### Of course, gotta make sure it's the right shape.

In [118]:
df2.shape

(569, 11)

#### Saving our final table to a csv.

In [119]:
df2.to_csv(r'/Users/stephanie/Desktop/prison-env/Executed_Offenders_Last_Statements.csv', index = False, header=True)

## Foreword:
Thank you for reading along! I hope you had as much fun reading as I did working on this! Please feel free to reach out if you have any questions or suggestions, thank you!